In [1]:
import os
import sys
import tqdm
import h5py

import numpy as np
import pandas as pd

from joblib import Parallel, delayed

import pathlib
import shutil
import random

# Initiating a dataframe

In [ ]:
df_map = pd.DataFrame()

In [ ]:
path_skel_folder = "/share/data/temp/athira/Tierpsy_skeleton_files_may15/"
(_,_,fnames) = next(os.walk(path_skel_folder))

In [ ]:
df_map['fname_tierpsy'] = fnames

In [ ]:
len(df_map.index)

# Mapping

In [ ]:
path_to_meta_toxtrac = "/share/data/longterm/2/Jerneja_Behaviour/COMBINED_BehaviourDataSet_2018/dataframes/common_data.pickle"
df_meta_toxtrac = pd.read_pickle(path_to_meta_toxtrac)
# list(df_meta_toxtrac.columns)


In [ ]:
df_meta_toxtrac['dataframepath'][0]

In [113]:
# Path to modified skeleton files - these hdf5 files has additional datasets in them. 
path_skel_folder = "/share/data/temp/athira/Tierpsy_skeleton_files_may15/" 
path_to_skel_files = [os.path.join(root,name) for root,dirs,filenames in os.walk(path_skel_folder) 
                                              for name in filenames]

print(f"Number of skeleton files in the new folder : {len(path_to_skel_files)} ")

Number of skeleton files in the new folder : 4087 


In [ ]:
def correct_drugname(data_folder,skel_fnames):
    
    import re
    import difflib
    # True (standardized) drug names taken from Jerneja's plot diagrams
    drug_names = ['AA', 'Dopamine','Fluoxetine','Methiothepin', 'None', 'Octopamine', 'Serotonin',
                   'Tyramine', 'Alphamethyl', 'Chlorpromazine', 'Clomipramine','Imipramine',  
                   'Mianserin','Paroxetine', 'Phentolamine', 'Quinpirole', 'Raclopride']

    # according to the file naming convention, the position of drug name is after the 5th underscore
    drug_pos = 4

   

    for fname in skel_fnames:

        
        old_path = os.path.join(data_folder,fname)
        # get only the alpha (name and not conc) part of the drug label
        drug_conc = re.findall(r"[^\W\d_]+|\d+", fname.split('_')[drug_pos])
        
        # QUICKFIX : 'ascorbicacid vs AA'
        drug = fname.split('_')[drug_pos]
        # find the closest match of the drug name from the list "drug_names"         
        true_drug_name = difflib.get_close_matches(drug, drug_names, n = 1, cutoff= 0.3)
        # replace the old spelling with the closest match 
        try:
                        
            if true_drug_name[0] == 'Quinpirole':
                # replace the old spelling with the closest match 
                fname_new = fname.replace(drug_conc[0],true_drug_name[0])
            else:
                fname_new = fname.replace(drug,true_drug_name[0])
#             fname_new = fname.replace(drug_conc[0],true_drug_name[0])
               
        except IndexError :
            fname_new = fname.replace(drug_conc[0],'AA')
        finally:
            new_path = os.path.join(data_folder, str(fname_new))
            # rename the file
        return new_path

In [ ]:
# Iterate through the rows

def map_filenames(ind, meta_row):
#     for index, row in df_meta_stimuli.iterrows():
    
    dict_stimuli = {}
    
    file_path = meta_row['dataframepath']

    # Alter the path name to skeleton style
    skel_file_path = os.path.split(file_path)[0]
    
    skel_file_path = skel_file_path.split('Exp_')[1]
    skel_file_path_ninv = f"{skel_file_path}_skeletons.hdf5"
    skel_file_path_ninv = correct_drugname(path_skel_folder,[skel_file_path_ninv])


    #Check if exists
    if skel_file_path_ninv in path_to_skel_files:
        dict_stimuli['skeletonpath'] = skel_file_path_ninv
    else :
        # Account for INVERTED tag
        skel_file_path_inv = f"{skel_file_path}_INVERTED_skeletons.hdf5"
        skel_file_path_inv = correct_drugname(path_skel_folder,[skel_file_path_inv])
        # Check if inverted exists
        if skel_file_path_inv in path_to_skel_files:
            dict_stimuli['skeletonpath'] = skel_file_path_inv
    
    if dict_stimuli:
        dict_stimuli['toxtracpath'] = file_path
        
        skeletons_store = h5py.File(dict_stimuli['skeletonpath'], 'r')
        dict_stimuli['quality'] = skeletons_store['quality'][()]
        dict_stimuli['n_frames'] = skeletons_store['n_frames'][()]
        skeletons_store.close()

        return dict_stimuli


        

In [ ]:
# Iterate through the rows

def map_timestamps(ind, meta_row):
#     for index, row in df_meta_stimuli.iterrows():
    
    dict_stimuli = {}
    
    file_path = meta_row['dataframepath']

    # Alter the path name to skeleton style
    skel_file_path = os.path.split(file_path)[0]
    
    skel_file_path = skel_file_path.split('Exp_')[1]
    skel_file_path_tags = skel_file_path.split('_')
    toxtrac_date_tag = skel_file_path_tags[0]
    toxtrac_time_tag = skel_file_path_tags[1]
    skel_file_path_ninv = f"{skel_file_path}_skeletons.hdf5"
    skel_file_path_ninv = correct_drugname(path_skel_folder,[skel_file_path_ninv])


    #Check if exists
    if skel_file_path_ninv in path_to_skel_files:
        dict_stimuli['skeletonpath'] = skel_file_path_ninv
    else :
        # Account for INVERTED tag
        skel_file_path_inv = f"{skel_file_path}_INVERTED_skeletons.hdf5"
        skel_file_path_inv = correct_drugname(path_skel_folder,[skel_file_path_inv])
        # Check if inverted exists
        if skel_file_path_inv in path_to_skel_files:
            dict_stimuli['skeletonpath'] = skel_file_path_inv
    
    if not dict_stimuli:
        
        try_path = [path for path in path_to_skel_files 
                                        if f"{path.split('_')[0]}_{path.split('_')[1]}" == f"{toxtrac_date_tag}_{toxtrac_time_tag}"]
        if try_path :
            dict_stimuli['skeletonpath'] = try_path[0]
            
    if dict_stimuli:
        dict_stimuli['toxtracpath'] = file_path
        
        skeletons_store = h5py.File(dict_stimuli['skeletonpath'], 'r')
        dict_stimuli['quality'] = skeletons_store['quality'][()]
        dict_stimuli['n_frames'] = skeletons_store['n_frames'][()]
        skeletons_store.close()

        return dict_stimuli


        

In [ ]:
dict_stimuli_list = Parallel(n_jobs=30, verbose = 5)(delayed(map_timestamps)(index, row) for index, row in df_meta_toxtrac.iterrows())

In [ ]:
print(len(dict_stimuli_list))

In [ ]:
valid_list = [x for x in dict_stimuli_list if x]

In [ ]:
len(valid_list)

In [ ]:
df_true_map = pd.DataFrame(valid_list) 

In [ ]:
test_ind = random.randint(0,len(df_true_map.index))

In [ ]:
df_true_map.iloc[test_ind]['skeletonpath']

In [ ]:
df_true_map.iloc[test_ind]['toxtracpath']

In [ ]:
df_true_map.to_pickle('/share/data/temp/athira/filenames_mapping_v2.pickle')

## Test

In [109]:
df_true_map = pd.read_pickle('/share/data/temp/athira/filenames_mapping.pickle')

In [110]:
len(df_true_map.skeletonpath.unique())

3618

In [ ]:
df_features_combined = pd.read_pickle('/share/data/temp/athira/June10_features_combined.pickle')

In [ ]:
len(df_features_combined.filename.unique())

In [111]:
df_true_map_sub = df_true_map[(df_true_map['quality']>40) & (df_true_map['n_frames']>200)]

In [112]:
len(df_true_map_sub.skeletonpath.unique())

2099

In [ ]:
new_list = list(set(df_features_combined.filename).difference(df_true_map_sub.skeletonpath))

In [ ]:
len(new_list)

In [ ]:
# for fname in new_list:
    
#     file_path = os.path.join(path_skel_folder,fname)
#     skeletons_store = h5py.File(file_path, 'r')

#     print(skeletons_store['quality'][()],skeletons_store['n_frames'][()])

#     skeletons_store.close()

    

In [ ]:
true_toxtrac_filenames = [path.split('/')[-2] for path in list(df_meta_toxtrac['dataframepath'])]


In [ ]:
matched_toxtrac_filenames = [path.split('/')[-2] for path in list(df_true_map_sub ['toxtracpath'])]


In [ ]:
len(true_toxtrac_filenames)

In [ ]:
len(matched_toxtrac_filenames)

In [ ]:
len(df_true_map_sub.index)

In [ ]:
true_tierpsy_filenames = [f"Exp_{path.split('_skeletons')[0]}" for path in list(df_features_combined.filename.unique())]

In [ ]:
missing_list = list(set(true_tierpsy_filenames)-  set(matched_toxtrac_filenames))

In [ ]:
len(missing_list)

In [ ]:
import difflib

In [ ]:
drugs_mispelled = []
for missing_path in missing_list:
    
    wrong_toxtrac_spelling = difflib.get_close_matches(missing_path, true_toxtrac_filenames, n= 1, cutoff= 0.3)
    mispelled_drug = wrong_toxtrac_spelling[0].split('_')[5]
    drugs_mispelled.append(mispelled_drug)
    

In [ ]:
len(drugs_mispelled)

In [ ]:
set(drugs_mispelled)

In [ ]:
len(set(drugs_mispelled))

# Map again

In [30]:
df_features_combined = pd.read_pickle('/share/data/temp/athira/June10_features_combined.pickle')

In [108]:
len(df_features_combined.filename.unique())

2308

In [28]:
path_to_meta_toxtrac = "/share/data/longterm/2/Jerneja_Behaviour/COMBINED_BehaviourDataSet_2018/dataframes/common_data.pickle"
df_meta_toxtrac = pd.read_pickle(path_to_meta_toxtrac)

In [31]:
df_tierpsy_filenames = pd.DataFrame(df_features_combined['filename'].unique(), columns=['filename'])
df_toxtrac_filenames = pd.DataFrame(df_meta_toxtrac['dataframepath'])

In [32]:
print(len(df_tierpsy_filenames.index), len(df_toxtrac_filenames.index))

2308 5167


In [ ]:
split_cols = df_tierpsy_filenames.filename.str.split('_', n = 8, expand = True)
df_tierpsy_filenames['date'] = split_cols[0]
df_tierpsy_filenames['time'] = split_cols[1]
df_tierpsy_filenames['drug'] = split_cols[4]

In [ ]:
df_tierpsy_filenames

In [33]:
split_cols2 = df_toxtrac_filenames.dataframepath.str.split('/Exp_', n = 1, expand = True)
df_toxtrac_filenames['test'] = split_cols2[1]
split_cols2 = df_toxtrac_filenames.test.str.split('/', n = 1, expand = True)
df_toxtrac_filenames['test'] = split_cols2[0]

In [34]:
df_toxtrac_filenames

,dataframepath,test
0,/share/data/longterm/2/Jernejas_stuff/COMBINED...,20180813_080353_1_5m0s_None_None_None
1,/share/data/longterm/2/Jernejas_stuff/COMBINED...,20180808_133433_1_15m0s_phentolamine_None_None
2,/share/data/longterm/2/Jernejas_stuff/COMBINED...,20180718_104634_1_5m0s_imipramine10_None_None
3,/share/data/longterm/2/Jernejas_stuff/COMBINED...,20180807_192540_1_5m0s_phentolamine_None_None
4,/share/data/longterm/2/Jernejas_stuff/COMBINED...,20180706_100913_1_5m0s_chlopromazine01_None_None
...,...,...
5162,/share/data/longterm/2/Jernejas_stuff/COMBINED...,20180717_141815_1_5m0s_serotonin100_None_None
5163,/share/data/longterm/2/Jernejas_stuff/COMBINED...,20180622_165851_1_5m0s_raclopride0025_None_None
5164,/share/data/longterm/2/Jernejas_stuff/COMBINED...,20180803_131932_1_15m0s_octopamine100_None_None
5165,/share/data/longterm/2/Jernejas_stuff/COMBINED...,20181211_154221_1_5m0s_None_None_Light


In [35]:
split_cols3 = df_toxtrac_filenames.test.str.split('_', n = 7, expand = True)
df_toxtrac_filenames['date'] = split_cols3[0]
df_toxtrac_filenames['time'] = split_cols3[1]
df_toxtrac_filenames['drug'] = split_cols3[4]
df_toxtrac_filenames

,dataframepath,test,date,time,drug
0,/share/data/longterm/2/Jernejas_stuff/COMBINED...,20180813_080353_1_5m0s_None_None_None,20180813,080353,None
1,/share/data/longterm/2/Jernejas_stuff/COMBINED...,20180808_133433_1_15m0s_phentolamine_None_None,20180808,133433,phentolamine
2,/share/data/longterm/2/Jernejas_stuff/COMBINED...,20180718_104634_1_5m0s_imipramine10_None_None,20180718,104634,imipramine10
3,/share/data/longterm/2/Jernejas_stuff/COMBINED...,20180807_192540_1_5m0s_phentolamine_None_None,20180807,192540,phentolamine
4,/share/data/longterm/2/Jernejas_stuff/COMBINED...,20180706_100913_1_5m0s_chlopromazine01_None_None,20180706,100913,chlopromazine01
...,...,...,...,...,...
5162,/share/data/longterm/2/Jernejas_stuff/COMBINED...,20180717_141815_1_5m0s_serotonin100_None_None,20180717,141815,serotonin100
5163,/share/data/longterm/2/Jernejas_stuff/COMBINED...,20180622_165851_1_5m0s_raclopride0025_None_None,20180622,165851,raclopride0025
5164,/share/data/longterm/2/Jernejas_stuff/COMBINED...,20180803_131932_1_15m0s_octopamine100_None_None,20180803,131932,octopamine100
5165,/share/data/longterm/2/Jernejas_stuff/COMBINED...,20181211_154221_1_5m0s_None_None_Light,20181211,154221,None


In [ ]:
new_df = pd.merge(df_tierpsy_filenames, df_toxtrac_filenames,  how='left', left_on=['date', 'time'], right_on = ['date', 'time'])

In [ ]:
len(new_df.index)

In [ ]:
import re
import difflib
# True (standardized) drug names taken from Jerneja's plot diagrams
drug_names = ['AA', 'Dopamine','Fluoxetine','Methiothepin', 'None', 'Octopamine', 'Serotonin',
               'Tyramine', 'Alphamethyl', 'Chlorpromazine', 'Clomipramine','Imipramine',  
               'Mianserin','Paroxetine', 'Phentolamine', 'Quinpirole', 'Raclopride']

In [ ]:
new_df_drugs = new_df.filter(like='drug')

In [ ]:
non_matching_drugs = new_df_drugs[new_df_drugs['drug_x'] != new_df_drugs['drug_y']].str.

In [ ]:
non_matching_drugs

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(40,5))
non_matching_drugs.drug_x.hist(ax=ax,histtype='bar')

In [ ]:
non_matching_drugs.assign(corr_drug=lambda x: x.drug_y)

# ToxTrac matching with Jerneja's tierpsy

In [134]:
skel_data_folder = "/share/data/longterm/2/Jerneja_Behaviour/Tierpsy_skeleton_files_jerneja/"
skel_files = [os.path.join(root,name) for root,dirs,files in os.walk(skel_data_folder) 
              for name in files if name.endswith('skeletons.hdf5')]

print(f"Total number of hdf5 skeleton files:{len(skel_files)}")

Total number of hdf5 skeleton files:4286


In [135]:
from glob import glob

In [136]:
danielfiles = glob(skel_data_folder+"/**/*skeletons.hdf5", recursive = True)

In [137]:
len(danielfiles)

4286

In [138]:
skel_filenames = pd.DataFrame(danielfiles, columns=['filepaths'])

In [139]:

skel_filenames.iloc[0].values

array(['/share/data/longterm/2/Jerneja_Behaviour/Tierpsy_skeleton_files_jerneja/20180822_2/Results/20180822_153547_1_15m0s_None_None_None_skeletons.hdf5'],
      dtype=object)

In [140]:
skel_file_df = pd.DataFrame(data = skel_filenames.filepaths.apply(lambda x:os.path.basename(x)).str.split('_', expand = True).values,
                            columns = ["date", "time", "crowdsize", "duration", "drug","genetics","stim","none","none2"])

In [141]:
skel_file_df

,date,time,crowdsize,duration,drug,genetics,stim,none,none2
0,20180822,153547,1,15m0s,None,None,None,skeletons.hdf5,None
1,20180822,171241,1,5m0s,None,None,None,skeletons.hdf5,None
2,20180822,171741,1,5m0s,alphamethyl,None,None,skeletons.hdf5,None
3,20180822,144548,1,5m0s,mianserin,None,None,skeletons.hdf5,None
4,20180822,155728,1,5m0s,mianserin,None,None,skeletons.hdf5,None
...,...,...,...,...,...,...,...,...,...
4281,20180807,170302,1,15m0s,Dopamine,None,None,skeletons.hdf5,None
4282,20180807,162642,1,15m0s,None,None,None,skeletons.hdf5,None
4283,20180807,184326,1,5m0s,phentolamine,None,None,skeletons.hdf5,None
4284,20180807,125255,1,5m0s,phentolamine,None,None,skeletons.hdf5,None


In [142]:
skel_file_df["inverted"] = skel_file_df["none"].str.lower() == "inverted"

In [143]:
sum(skel_file_df.inverted)

1242

In [144]:
pd.DataFrame(data = df_meta_toxtrac['dataframepath'].str.split('/').apply(lambda x:x[-2]).str.split("_", expand = True),
                                   columns = ["date", "time", "crowdsize", "duration", "drug","genetics","stim"])

,date,time,crowdsize,duration,drug,genetics,stim
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
5162,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5163,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5164,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5165,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [55]:
temp_df = df_meta_toxtrac["dataframepath"].str.split("/").apply(lambda x: x[-2]).str.split("_", expand = True)

In [145]:
tox_file_df = pd.DataFrame(data = temp_df.values, columns = ["exp","date", "time", "crowdsize", "duration", "drug","genetics","stim"])

In [146]:
tox_file_df = tox_file_df[["date", "time", "crowdsize", "duration", "drug","genetics","stim"]]

In [147]:
skel_file_df = skel_file_df[tox_file_df.columns]

In [148]:
skel_file_df

,date,time,crowdsize,duration,drug,genetics,stim
0,20180822,153547,1,15m0s,None,None,None
1,20180822,171241,1,5m0s,None,None,None
2,20180822,171741,1,5m0s,alphamethyl,None,None
3,20180822,144548,1,5m0s,mianserin,None,None
4,20180822,155728,1,5m0s,mianserin,None,None
...,...,...,...,...,...,...,...
4281,20180807,170302,1,15m0s,Dopamine,None,None
4282,20180807,162642,1,15m0s,None,None,None
4283,20180807,184326,1,5m0s,phentolamine,None,None
4284,20180807,125255,1,5m0s,phentolamine,None,None


In [149]:
skel_file_df.stim.unique()

array(['None', 'Light', 'None (2)'], dtype=object)

In [150]:
skel_file_df[skel_file_df.stim == 'None (2)']

,date,time,crowdsize,duration,drug,genetics,stim
1414,20180806,184158,1,5m0s,None,None,None (2)
1436,20180806,174455,1,15m0s,tyramine,None,None (2)
1440,20180806,191759,1,5m0s,tyramine,None,None (2)
1459,20180806,185648,1,15m0s,tyramine,None,None (2)
1470,20180806,183657,1,5m0s,None,None,None (2)
1476,20180806,192300,1,5m0s,tyramine,None,None (2)
1489,20180806,184659,1,5m0s,None,None,None (2)
1490,20180806,144245,1,15m0s,None,None,None (2)
1494,20180806,191258,1,5m0s,tyramine,None,None (2)
2762,20180806,184650,1,5m0s,tyramine,None,None (2)


In [151]:
tox_file_df

,date,time,crowdsize,duration,drug,genetics,stim
0,20180813,080353,1,5m0s,None,None,None
1,20180808,133433,1,15m0s,phentolamine,None,None
2,20180718,104634,1,5m0s,imipramine10,None,None
3,20180807,192540,1,5m0s,phentolamine,None,None
4,20180706,100913,1,5m0s,chlopromazine01,None,None
...,...,...,...,...,...,...,...
5162,20180717,141815,1,5m0s,serotonin100,None,None
5163,20180622,165851,1,5m0s,raclopride0025,None,None
5164,20180803,131932,1,15m0s,octopamine100,None,None
5165,20181211,154221,1,5m0s,None,None,Light


In [152]:
tox_file_df.stim.unique()

array(['None', 'Light', 'None (2)'], dtype=object)

In [153]:
tox_file_df[tox_file_df.stim == 'None (2)']

,date,time,crowdsize,duration,drug,genetics,stim
461,20180807,124253,1,5m0s,Dopamine,None,None (2)
761,20180806,184149,1,5m0s,tyramine,None,None (2)
917,20180807,134016,1,15m0s,None,None,None (2)
1305,20180806,185626,1,15m0s,None,None,None (2)
3585,20180806,183648,1,5m0s,tyramine,None,None (2)
4924,20180806,184650,1,5m0s,tyramine,None,None (2)


In [154]:
skelcounts = skel_file_df.groupby(["date","time"]).count()

In [155]:
skelcounts[skelcounts["drug"] > 1]

crowdsize  duration  drug  genetics  stim
date     time                                             
20180705 084537          2         2     2         2     2
         085602          2         2     2         2     2
         100059          2         2     2         2     2
         100606          2         2     2         2     2
         112709          2         2     2         2     2
...                    ...       ...   ...       ...   ...
20181214 102312          2         2     2         2     2
         102316          2         2     2         2     2
         102854          2         2     2         2     2
         102857          2         2     2         2     2
         102900          2         2     2         2     2

[349 rows x 5 columns]

In [156]:
skelcounts.crowdsize.unique()

array([1, 2, 4, 3])

In [157]:
toxcounts = tox_file_df.groupby(["date","time"]).count()

In [158]:
toxcounts[toxcounts["genetics"] > 1].shape

(219, 5)

In [159]:
toxcounts.crowdsize.unique()

array([1, 2, 3])

In [160]:
toxcounts.max()

crowdsize    3
duration     3
drug         3
genetics     3
stim         3
dtype: int64

In [161]:
merged_df = pd.merge(skel_file_df, tox_file_df,  how='left', left_on=['date', 'time'], right_on = ['date', 'time'], suffixes=('skel', 'tox'))

In [162]:
len(merged_df.index)

4628

In [163]:
merged_df

,date,time,crowdsizeskel,durationskel,drugskel,geneticsskel,stimskel,crowdsizetox,durationtox,drugtox,geneticstox,stimtox
0,20180822,153547,1,15m0s,None,None,None,1,15m0s,None,None,None
1,20180822,171241,1,5m0s,None,None,None,1,5m0s,mianserin,None,None
2,20180822,171741,1,5m0s,alphamethyl,None,None,1,5m0s,alphamethyl,None,None
3,20180822,144548,1,5m0s,mianserin,None,None,1,5m0s,mianserin,None,None
4,20180822,155728,1,5m0s,mianserin,None,None,1,5m0s,mianserin,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...
4623,20180807,162642,1,15m0s,None,None,None,1,15m0s,None,None,None
4624,20180807,184326,1,5m0s,phentolamine,None,None,NaN,NaN,NaN,NaN,NaN
4625,20180807,125255,1,5m0s,phentolamine,None,None,1,5m0s,Dopamine,None,None
4626,20180807,125255,1,5m0s,phentolamine,None,None,1,5m0s,phentolamine,None,None


In [164]:
merged_df.drugtox.isnull().sum()

167

In [165]:
merged_duplicate_count = merged_df.groupby(['date','time']).count()

In [166]:
merged_duplicate_count[merged_duplicate_count['crowdsizeskel'] > 1]

crowdsizeskel  durationskel  drugskel  geneticsskel  \
date     time                                                          
20180705 084537              4             4         4             4   
         085602              4             4         4             4   
         100059              4             4         4             4   
         100606              4             4         4             4   
         112709              4             4         4             4   
...                        ...           ...       ...           ...   
20181214 102312              2             2         2             2   
         102316              2             2         2             2   
         102854              2             2         2             2   
         102857              2             2         2             2   
         102900              2             2         2             2   

                 stimskel  crowdsizetox  durationtox  drugtox  geneticstox  \
date     time                                                                
20180705 084537         4             4            4        4            4   
         085602         4             4            4        4            4   
         100059         4             4            4        4            4   
         100606         4             4            4        4            4   
         112709         4             4            4        4            4   
...                   ...           ...          ...      ...          ...   
20181214 102312         2             2            2        2            2   
         102316         2             2            2        2            2   
         102854         2             2            2        2            2   
         102857         2             2            2        2            2   
         102900         2             2            2        2            2   

                 stimtox  
date     time             
20180705 084537        4  
         085602        4  
         100059        4  
         100606        4  
         112709        4  
...                  ...  
20181214 102312        2  
         102316        2  
         102854        2  
         102857        2  
         102900        2  

[376 rows x 10 columns]

Let us see what happens if we try to match the Tierpsy and ToxTrac files based on date,time, drug and stim

In [174]:
merged_adv_df = pd.merge(skel_file_df, tox_file_df,  how='left', left_on=['date', 'time', 'drug', 'stim'], right_on = ['date', 'time', 'drug', 'stim'], suffixes=('skel', 'tox'))

In [175]:
merged_adv_df

,date,time,crowdsizeskel,durationskel,drug,geneticsskel,stim,crowdsizetox,durationtox,geneticstox
0,20180822,153547,1,15m0s,None,None,None,1,15m0s,None
1,20180822,171241,1,5m0s,None,None,None,NaN,NaN,NaN
2,20180822,171741,1,5m0s,alphamethyl,None,None,1,5m0s,None
3,20180822,144548,1,5m0s,mianserin,None,None,1,5m0s,None
4,20180822,155728,1,5m0s,mianserin,None,None,1,5m0s,None
...,...,...,...,...,...,...,...,...,...,...
4305,20180807,170302,1,15m0s,Dopamine,None,None,1,15m0s,None
4306,20180807,162642,1,15m0s,None,None,None,1,15m0s,None
4307,20180807,184326,1,5m0s,phentolamine,None,None,NaN,NaN,NaN
4308,20180807,125255,1,5m0s,phentolamine,None,None,1,5m0s,None


Using pandas outer merge to see if missing files can easily be traced. 
- This can be done to see, in total, how many files cannot be matched between the Tierpsy and ToxTrac taking into account all the columns

In [170]:
df_merged_outer = pd.merge(skel_file_df, tox_file_df,  how='outer', indicator=True)

In [171]:
# Only exists in Tierpsy
df_merged_outer[df_merged_outer._merge == 'left_only']

,date,time,crowdsize,duration,drug,genetics,stim,_merge
1,20180822,171241,1,5m0s,None,None,None,left_only
7,20180822,172245,1,5m0s,None,None,None,left_only
10,20180822,171129,1,1m0s,None,None,None,left_only
17,20180822,171742,1,5m0s,None,None,None,left_only
18,20180822,171119,1,1m0s,None,None,None,left_only
...,...,...,...,...,...,...,...,...
4122,20180717,134402,1,5m0s,None,None,None,left_only
4138,20180717,133857,1,5m0s,None,None,None,left_only
4155,20180717,133352,1,5m0s,None,None,None,left_only
4303,20180807,124253,1,5m0s,Dopamine,None,None,left_only


In [173]:
# Only exists in ToxTrac
df_merged_outer[df_merged_outer._merge == 'right_only']

,date,time,crowdsize,duration,drug,genetics,stim,_merge
4310,20180807,192540,1,5m0s,phentolamine,None,None,right_only
4311,20180622,181201,1,5m0s,raclopride0025,None,None,right_only
4312,20180921,110323,1,5m0s,None,None,None,right_only
4313,20180803,152639,1,5m0s,tyramine10,None,None,right_only
4314,20180622,173514,1,5m0s,None,None,None,right_only
...,...,...,...,...,...,...,...,...
5675,20181204,180100,1,5m0s,None,None,Light,right_only
5676,20180621,110703,1,5m0s,None,None,None,right_only
5677,20180622,165851,1,5m0s,raclopride0025,None,None,right_only
5678,20180803,131932,1,15m0s,octopamine100,None,None,right_only


Let us see which ones have a mismatch in drugs

- When the tierpsy files and toxtrax are matched based on the date ad time, there are caes where the drugs are not matching
- Let us print it out and see how many there are!

In [118]:
merged_df[merged_df.drugskel != merged_df.drugtox]

,date,time,crowdsizeskel,durationskel,drugskel,geneticsskel,stimskel,crowdsizetox,durationtox,drugtox,geneticstox,stimtox
1,20180822,171241,1,5m0s,None,None,None,1,5m0s,mianserin,None,None
7,20180822,172245,1,5m0s,None,None,None,NaN,NaN,NaN,NaN,NaN
10,20180822,171129,1,1m0s,None,None,None,NaN,NaN,NaN,NaN,NaN
14,20180822,142027,1,15m0s,mianserin,None,None,1,15m0s,None,None,None
18,20180822,171742,1,5m0s,None,None,None,1,5m0s,mianserin,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...
4597,20180807,124253,1,5m0s,Dopamine,None,None (2),1,5m0s,phentolamine,None,None
4614,20180807,125255,1,5m0s,Dopamine,None,None,1,5m0s,phentolamine,None,None
4620,20180807,124253,1,5m0s,Dopamine,None,None,1,5m0s,phentolamine,None,None
4624,20180807,184326,1,5m0s,phentolamine,None,None,NaN,NaN,NaN,NaN,NaN


Now let us try to see how many missing files exist if we join just based on the date time drug and stimulus

In [178]:
df_merged_outer_dtds = pd.merge(skel_file_df, tox_file_df,  how='outer', on= ['date','time','drug','stim'], indicator=True)

In [179]:
df_merged_outer_dtds[df_merged_outer_dtds._merge=='left_only']

,date,time,crowdsize_x,duration_x,drug,genetics_x,stim,crowdsize_y,duration_y,genetics_y,_merge
1,20180822,171241,1,5m0s,None,None,None,NaN,NaN,NaN,left_only
7,20180822,172245,1,5m0s,None,None,None,NaN,NaN,NaN,left_only
10,20180822,171129,1,1m0s,None,None,None,NaN,NaN,NaN,left_only
17,20180822,171742,1,5m0s,None,None,None,NaN,NaN,NaN,left_only
18,20180822,171119,1,1m0s,None,None,None,NaN,NaN,NaN,left_only
...,...,...,...,...,...,...,...,...,...,...,...
4122,20180717,134402,1,5m0s,None,None,None,NaN,NaN,NaN,left_only
4138,20180717,133857,1,5m0s,None,None,None,NaN,NaN,NaN,left_only
4155,20180717,133352,1,5m0s,None,None,None,NaN,NaN,NaN,left_only
4303,20180807,124253,1,5m0s,Dopamine,None,None,NaN,NaN,NaN,left_only


In [180]:
df_merged_outer_dtds[df_merged_outer_dtds._merge=='right_only']

,date,time,crowdsize_x,duration_x,drug,genetics_x,stim,crowdsize_y,duration_y,genetics_y,_merge
4310,20180807,192540,NaN,NaN,phentolamine,NaN,None,1,5m0s,None,right_only
4311,20180622,181201,NaN,NaN,raclopride0025,NaN,None,1,5m0s,None,right_only
4312,20180921,110323,NaN,NaN,None,NaN,None,1,5m0s,None,right_only
4313,20180803,152639,NaN,NaN,tyramine10,NaN,None,1,5m0s,None,right_only
4314,20180622,173514,NaN,NaN,None,NaN,None,1,5m0s,None,right_only
...,...,...,...,...,...,...,...,...,...,...,...
5543,20181204,180100,NaN,NaN,None,NaN,Light,1,5m0s,None,right_only
5544,20180621,110703,NaN,NaN,None,NaN,None,1,5m0s,None,right_only
5545,20180622,165851,NaN,NaN,raclopride0025,NaN,None,1,5m0s,None,right_only
5546,20180803,131932,NaN,NaN,octopamine100,NaN,None,1,15m0s,None,right_only


- From the above column, we can spot cases where the mismatches are due to duplicates existing in either the tierpsy files or the toxtarc files (Duplicates in terms of date and time, so more like experiments which were run simultaneously).
- If the mismatches were to be grouped on the basis of date and time, we could see if duplicates exists in both tierpsy and toxtrac

In [122]:
count_drug_mismatch = merged_df[merged_df.drugskel != merged_df.drugtox].groupby(['date','time']).count()

In [125]:
count_drug_mismatch

crowdsizeskel  durationskel  drugskel  geneticsskel  \
date     time                                                          
20180705 084537              2             2         2             2   
         085602              2             2         2             2   
         100059              2             2         2             2   
         100606              2             2         2             2   
         112709              2             2         2             2   
...                        ...           ...       ...           ...   
20181121 171922              1             1         1             1   
20181128 162807              1             1         1             1   
         162818              1             1         1             1   
         171226              1             1         1             1   
20181214 090110              2             2         2             2   

                 stimskel  crowdsizetox  durationtox  drugtox  geneticstox  \
date     time                                                                
20180705 084537         2             2            2        2            2   
         085602         2             2            2        2            2   
         100059         2             2            2        2            2   
         100606         2             2            2        2            2   
         112709         2             2            2        2            2   
...                   ...           ...          ...      ...          ...   
20181121 171922         1             0            0        0            0   
20181128 162807         1             0            0        0            0   
         162818         1             0            0        0            0   
         171226         1             0            0        0            0   
20181214 090110         2             0            0        0            0   

                 stimtox  
date     time             
20180705 084537        2  
         085602        2  
         100059        2  
         100606        2  
         112709        2  
...                  ...  
20181121 171922        0  
20181128 162807        0  
         162818        0  
         171226        0  
20181214 090110        0  

[323 rows x 10 columns]

- The cases where there are unequal count for a date_time between Tierpsy and Toxtrac, these could be a possible case of missing files

In [127]:
count_missing = count_drug_mismatch[count_drug_mismatch['drugskel'] != count_drug_mismatch['drugtox']]

In [128]:
count_missing

crowdsizeskel  durationskel  drugskel  geneticsskel  \
date     time                                                          
20180705 124136              1             1         1             1   
20180706 115843              1             1         1             1   
20180709 121000              1             1         1             1   
20180710 114758              1             1         1             1   
         122511              1             1         1             1   
...                        ...           ...       ...           ...   
20181121 171922              1             1         1             1   
20181128 162807              1             1         1             1   
         162818              1             1         1             1   
         171226              1             1         1             1   
20181214 090110              2             2         2             2   

                 stimskel  crowdsizetox  durationtox  drugtox  geneticstox  \
date     time                                                                
20180705 124136         1             0            0        0            0   
20180706 115843         1             0            0        0            0   
20180709 121000         1             0            0        0            0   
20180710 114758         1             0            0        0            0   
         122511         1             0            0        0            0   
...                   ...           ...          ...      ...          ...   
20181121 171922         1             0            0        0            0   
20181128 162807         1             0            0        0            0   
         162818         1             0            0        0            0   
         171226         1             0            0        0            0   
20181214 090110         2             0            0        0            0   

                 stimtox  
date     time             
20180705 124136        0  
20180706 115843        0  
20180709 121000        0  
20180710 114758        0  
         122511        0  
...                  ...  
20181121 171922        0  
20181128 162807        0  
         162818        0  
         171226        0  
20181214 090110        0  

[163 rows x 10 columns]

## The List that Marios needs

So, Marios needs a list of (in .xls format) those cases where there matching by date maps wrong files/experiments.

- The obvious ones are those in which the drugs don't match
    - However, seems like atleast some of these are due to duplicates existing in one of the datasets
- 

In [183]:
# Match the Tierpsy and ToxTrac files based on Date and Time 
df_merged_dt = pd.merge(skel_file_df, tox_file_df,  how='left', 
                        left_on=['date', 'time'], right_on = ['date', 'time'], 
                        suffixes=('skel', 'tox'))
df_merged_dt 

,date,time,crowdsizeskel,durationskel,drugskel,geneticsskel,stimskel,crowdsizetox,durationtox,drugtox,geneticstox,stimtox
0,20180822,153547,1,15m0s,None,None,None,1,15m0s,None,None,None
1,20180822,171241,1,5m0s,None,None,None,1,5m0s,mianserin,None,None
2,20180822,171741,1,5m0s,alphamethyl,None,None,1,5m0s,alphamethyl,None,None
3,20180822,144548,1,5m0s,mianserin,None,None,1,5m0s,mianserin,None,None
4,20180822,155728,1,5m0s,mianserin,None,None,1,5m0s,mianserin,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...
4623,20180807,162642,1,15m0s,None,None,None,1,15m0s,None,None,None
4624,20180807,184326,1,5m0s,phentolamine,None,None,NaN,NaN,NaN,NaN,NaN
4625,20180807,125255,1,5m0s,phentolamine,None,None,1,5m0s,Dopamine,None,None
4626,20180807,125255,1,5m0s,phentolamine,None,None,1,5m0s,phentolamine,None,None


In [210]:
# Take those files where drugs aren't matching
df_merged_misdrug_dt = df_merged_dt[df_merged_dt.drugskel != df_merged_dt.drugtox]
df_merged_misdrug_dt

,date,time,crowdsizeskel,durationskel,drugskel,geneticsskel,stimskel,crowdsizetox,durationtox,drugtox,geneticstox,stimtox
1,20180822,171241,1,5m0s,None,None,None,1,5m0s,mianserin,None,None
7,20180822,172245,1,5m0s,None,None,None,NaN,NaN,NaN,NaN,NaN
10,20180822,171129,1,1m0s,None,None,None,NaN,NaN,NaN,NaN,NaN
14,20180822,142027,1,15m0s,mianserin,None,None,1,15m0s,None,None,None
18,20180822,171742,1,5m0s,None,None,None,1,5m0s,mianserin,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...
4597,20180807,124253,1,5m0s,Dopamine,None,None (2),1,5m0s,phentolamine,None,None
4614,20180807,125255,1,5m0s,Dopamine,None,None,1,5m0s,phentolamine,None,None
4620,20180807,124253,1,5m0s,Dopamine,None,None,1,5m0s,phentolamine,None,None
4624,20180807,184326,1,5m0s,phentolamine,None,None,NaN,NaN,NaN,NaN,NaN


In [212]:
# These are all the possible duplicates in the dataframe. 
test_df = df_merged_misdrug_dt[df_merged_misdrug_dt.duplicated(['date','time'], keep=False)].groupby(by=['date','time']).count()
# 
test_df[test_df.drugskel!=test_df.drugtox]

crowdsizeskel  durationskel  drugskel  geneticsskel  \
date     time                                                          
20180727 105121              2             2         2             2   
         135551              2             2         2             2   
         154649              2             2         2             2   
20181214 090110              2             2         2             2   

                 stimskel  crowdsizetox  durationtox  drugtox  geneticstox  \
date     time                                                                
20180727 105121         2             0            0        0            0   
         135551         2             0            0        0            0   
         154649         2             0            0        0            0   
20181214 090110         2             0            0        0            0   

                 stimtox  
date     time             
20180727 105121        0  
         135551        0  
         154649        0  
20181214 090110        0

Trying inner merge 

In [214]:
# Match the Tierpsy and ToxTrac files based on Date and Time 
df_innermerge_dt = pd.merge(skel_file_df, tox_file_df,  how='inner', 
                        left_on=['date', 'time'], right_on = ['date', 'time'], 
                        suffixes=('skel', 'tox'), sort=True)
df_innermerge_dt

,date,time,crowdsizeskel,durationskel,drugskel,geneticsskel,stimskel,crowdsizetox,durationtox,drugtox,geneticstox,stimtox
0,20180705,083019,1,15m0s,Fluoxetine10,None,None,1,15m0s,Fluoxetine10,None,None
1,20180705,083020,1,15m0s,None,None,None,1,15m0s,None,None,None
2,20180705,084537,1,5m0s,Fluoxetine10,None,None,1,5m0s,None,None,None
3,20180705,084537,1,5m0s,Fluoxetine10,None,None,1,5m0s,Fluoxetine10,None,None
4,20180705,084537,1,5m0s,None,None,None,1,5m0s,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...
4456,20181214,102854,1,5m0s,None,None,Light,1,5m0s,None,None,Light
4457,20181214,102857,1,5m0s,None,None,Light,1,5m0s,None,None,Light
4458,20181214,102857,1,5m0s,None,None,Light,1,5m0s,None,None,Light
4459,20181214,102900,1,5m0s,None,None,Light,1,5m0s,None,None,Light


In [216]:
df_innermerge_dt_mismatchdrugs = df_innermerge_dt[df_innermerge_dt.drugskel!=df_innermerge_dt.drugtox]

In [217]:
df_innermerge_dt_mismatchdrugs

,date,time,crowdsizeskel,durationskel,drugskel,geneticsskel,stimskel,crowdsizetox,durationtox,drugtox,geneticstox,stimtox
2,20180705,084537,1,5m0s,Fluoxetine10,None,None,1,5m0s,None,None,None
5,20180705,084537,1,5m0s,None,None,None,1,5m0s,Fluoxetine10,None,None
9,20180705,085602,1,5m0s,Fluoxetine10,None,None,1,5m0s,None,None,None
12,20180705,085602,1,5m0s,None,None,None,1,5m0s,Fluoxetine10,None,None
25,20180705,100059,1,5m0s,Fluoxetine10,None,None,1,5m0s,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...
3882,20180921,143732,1,5m0s,AA,None,Light,1,5m0s,Dopamine,None,Light
3988,20180926,162400,1,5m0s,AA,None,Light,1,5m0s,Dopamine,None,Light
3991,20180926,162400,1,5m0s,Dopamine,None,Light,1,5m0s,AA,None,Light
4023,20180926,175323,1,5m0s,Dopamine,None,Light,1,5m0s,AA,None,Light


In [225]:
df_duplicates = df_innermerge_dt_mismatchdrugs[df_innermerge_dt_mismatchdrugs.duplicated(['date','time'], keep=False)]
df_duplicates

,date,time,crowdsizeskel,durationskel,drugskel,geneticsskel,stimskel,crowdsizetox,durationtox,drugtox,geneticstox,stimtox
2,20180705,084537,1,5m0s,Fluoxetine10,None,None,1,5m0s,None,None,None
5,20180705,084537,1,5m0s,None,None,None,1,5m0s,Fluoxetine10,None,None
9,20180705,085602,1,5m0s,Fluoxetine10,None,None,1,5m0s,None,None,None
12,20180705,085602,1,5m0s,None,None,None,1,5m0s,Fluoxetine10,None,None
25,20180705,100059,1,5m0s,Fluoxetine10,None,None,1,5m0s,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...
3882,20180921,143732,1,5m0s,AA,None,Light,1,5m0s,Dopamine,None,Light
3988,20180926,162400,1,5m0s,AA,None,Light,1,5m0s,Dopamine,None,Light
3991,20180926,162400,1,5m0s,Dopamine,None,Light,1,5m0s,AA,None,Light
4023,20180926,175323,1,5m0s,Dopamine,None,Light,1,5m0s,AA,None,Light


In [234]:
marios_list = pd.merge(df_innermerge_dt_mismatchdrugs, df_duplicates, how='outer', on= ['date','time','drugskel','drugtox','stimskel','stimtox'], indicator=True)

In [231]:
marios_list[marios_list._merge == 'left_only']['_merge'] = 'non-duplicates'

/share/python-virtual-environments/ciona_behaviour/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [233]:
marios_list._merge.unique()

[both, left_only]
Categories (2, object): [both, left_only]

In [238]:
marios_list.to_excel('missing_files_data.xlsx')

In [237]:
!pip install openpyxl